# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面最底端")
        break
    last_height = new_height
    print('網頁更新中...')

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面最底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

tmp = 0

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)
    
    # 為了讓 github 能順利顯示
    tmp += 1
    if tmp == 10:
        break

----------------------------------------------------------------------

[0] 赴美上市突遭網路安全審查　滴滴出行否認洩露數據給美國

▲滴滴海外上市突遭中共安全審查。（圖／CFP）
記者鄭思楠／綜合報導
大陸網路叫車巨頭滴滴出行才剛赴美IPO，2日突遭官方實施網路安全審查，網傳涉及洩露敏感數據給美國。對此，滴滴副總裁李敏3日正面回應，滴滴的數據都存放在國內，絕無可能交給美國。
滴滴出行6月29日在美國市場IPO（首次公開募股）後，大陸網信辦2日突以「防範國家數據安全風險，維護國家安全」為由，對滴滴出行實施網路安全審查，期間並停止新用戶註冊。對此，大陸網路盛傳，滴滴洩露涉及國家安全的道路交通敏感數據給美國，因而攤上大事。
對此，滴滴副總裁李敏3日透過微博發文稱，看到網上有人惡意造謠說「滴滴在海外上市，把資料打包交給美國」。和眾多在海外上市的中國企業一樣，滴滴國內用戶的數據都存放在國內伺服器，「絕無可能把數據交給美國。」她強調，相關惡意造謠者雖然已經主動刪文，「但我們堅決起訴維權。」
滴滴出行官方微博隨後也轉發李敏的澄清文，呼籲「不傳謠，不信謠」！不過，隨後，依舊有網友質疑該澄清只提到「用戶數據」，沒提「道路數據」，李敏隨後在微博再次回應稱，「也包括道路數據，請不要惡意揣測。」
----------------------------------------------------------------------

[1] 8萬5千個工程師職缺　專家4步驟讓你輕鬆應對面試

▲半導體、面板等產業都相當缺人才，對此1111人力銀行也教該怎麼準備面試。（圖／取自免費圖庫Pixabay）
記者余弦妙 ／台北報導
受到新冠疫情影響，三級警戒防疫延長到7月12日，半導體、面板等科技大廠也都提升防疫層級，目前正值新鮮人求職旺季，過往深入企業面對面招攬新鮮人的情景已不復見，那到底該怎麼樣才能擠進人人稱羨的半導體業呢？專家就提出以下幾招讓你做好準備，輕鬆擠進年薪百萬的科技業。
1111人力銀行發言人黃若薇表示，使用1111主網關鍵字搜尋「工程師」出現近8萬5千筆工作機會，相較去年同期(2020年6月)呈現向上趨勢，增加一成左右，顯見疫情加速產業數位轉型，今年台灣就業市場仍以科技業為主要動力，整體徵才市場中，數